## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_DataBase.csv")
city_data_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tambopata,-12.7333,-69.1833,85.95,44,92,1.36,PE,2021-09-27 02:42:14
1,1,Yellowknife,62.4560,-114.3525,42.57,87,20,4.61,CA,2021-09-27 02:34:29
2,2,Hobart,-42.8794,147.3294,59.11,47,20,4.00,AU,2021-09-27 02:38:28
3,3,Puerto Ayora,-0.7393,-90.3518,69.22,90,87,6.91,EC,2021-09-27 02:38:11
4,4,Punta Arenas,-53.1500,-70.9167,41.11,81,0,12.66,CL,2021-09-27 02:37:33
5,5,Rikitea,-23.1203,-134.9692,73.65,72,33,19.60,PF,2021-09-27 02:38:14
6,6,Darhan,49.4867,105.9228,46.17,51,93,2.57,MN,2021-09-27 02:42:15
7,7,Kapaa,22.0752,-159.3190,84.18,75,40,7.00,US,2021-09-27 02:35:42
8,8,Arraial Do Cabo,-22.9661,-42.0278,71.08,87,86,22.15,BR,2021-09-27 02:42:16
9,9,Pevek,69.7008,170.3133,31.89,89,100,23.04,RU,2021-09-27 02:38:26


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("What is the minimum temperature you want in your trip? "))
max_temp = int(input("What is the maximum temperature you want in your trip? "))

What is the minimum temperature you want in your trip? 75
What is the maximum temperature you want in your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
preferred_cities.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tambopata,-12.7333,-69.1833,85.95,44,92,1.36,PE,2021-09-27 02:42:14
7,7,Kapaa,22.0752,-159.3190,84.18,75,40,7.00,US,2021-09-27 02:35:42
12,12,Vaini,-21.2000,-175.2000,77.16,78,75,5.75,TO,2021-09-27 02:39:49
13,13,Hilo,19.7297,-155.0900,80.33,86,90,12.66,US,2021-09-27 02:38:27
14,14,Pringsewu,-5.3582,104.9744,87.76,59,85,3.42,ID,2021-09-27 02:42:17
19,19,Davila,18.4726,120.5763,89.58,69,5,7.38,PH,2021-09-27 02:42:19
21,21,Atuona,-9.8000,-139.0333,78.08,73,5,18.81,PF,2021-09-27 02:38:12
24,24,Tatawin,32.9297,10.4518,76.08,66,9,2.53,TN,2021-09-27 02:42:20
30,30,Taoudenni,22.6783,-3.9836,89.82,12,7,8.70,ML,2021-09-27 02:39:45
39,39,Georgetown,5.4112,100.3354,84.13,76,20,1.01,MY,2021-09-27 02:36:56


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

Unnamed: 0    0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = preferred_cities.dropna()

# there were not any null values present so nothing changed
preferred_cities.count()

Unnamed: 0    168
City          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
Country       168
Date          168
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Tambopata,PE,85.95,-12.7333,-69.1833,
7,Kapaa,US,84.18,22.0752,-159.3190,
12,Vaini,TO,77.16,-21.2000,-175.2000,
13,Hilo,US,80.33,19.7297,-155.0900,
14,Pringsewu,ID,87.76,-5.3582,104.9744,
19,Davila,PH,89.58,18.4726,120.5763,
21,Atuona,PF,78.08,-9.8000,-139.0333,
24,Tatawin,TN,76.08,32.9297,10.4518,
30,Taoudenni,ML,89.82,22.6783,-3.9836,
39,Georgetown,MY,84.13,5.4112,100.3354,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [19]:
hotel_df.count()

City          168
Country       168
Max Temp      168
Lat           168
Lng           168
Hotel Name    168
dtype: int64

In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df["Hotel Name"] == ""]
# this code shows that there are 15 rows without hotel name
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df.count()
#this will show that we have 15 less rows simply by dropping the ones that couldn't find a hotel

City          157
Country       157
Max Temp      157
Lat           157
Lng           157
Hotel Name    157
dtype: int64

In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Kapaa,US,84.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Vaini,TO,77.16,-21.2000,-175.2000,Keleti Beach Resort
13,Hilo,US,80.33,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Pringsewu,ID,87.76,-5.3582,104.9744,Urban Style by Front One
19,Davila,PH,89.58,18.4726,120.5763,Blue Ridge Beach Resort
21,Atuona,PF,78.08,-9.8000,-139.0333,Villa Enata
24,Tatawin,TN,76.08,32.9297,10.4518,Sangho Privilege Tataouine
39,Georgetown,MY,84.13,5.4112,100.3354,Cititel Penang
45,Lodwar,KE,82.06,3.1191,35.5973,Solmar Gracious Hotel
53,Dakar,SN,79.68,14.6937,-17.4441,Redface Studios Meublés


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt></dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))